# Import

In [3]:
import pandas as pd
import numpy as np
import os
import sklearn
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from hyperopt import hp, tpe, fmin, Trials
from sklearn.model_selection import StratifiedKFold
import hyperopt
from sklearn.metrics import *
import time
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics 
import math
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import LeaveOneOut
import shap  # package used to calculate Shap values
import matplotlib.pylab as plt
from matplotlib import pyplot
from xgboost import plot_importance
import csv

import scipy
from scipy.stats import wilcoxon

# Boosting Based Pruning Class

In [ ]:
def WeightedError(Ct, X, y, w, N):
    """
    Calculate the training error weighted by sample weights 

    Parameters
    ----------
    Ct: a baaging estimator trained on full training data

    X : numpy array, shape = [n_samples, n_features]
              Training data
    y : list or numpy array, shape = [n_samples]
              Class labels
    
    w : numpy array, shape = [n_samples]
              boosting like sample weights
    N: int
        num of samples in training data

    Returns
    ----------
    eu : int
        weighted error of Ct on training data
    
    """

    y_pred=Ct.predict(X)
    eu= np.sum(np.where(y != y_pred, w, 0))/np.sum(w)
    return eu

def SelectBest(C, X, y, w, N):
    """
    Select the estimator with the mininal training error in pool_c

    Parameters
    ----------
    C: pool of baaging estimators trained on full training data

    X : numpy array, shape = [n_samples, n_features]
              Training data
    y : list or numpy array, shape = [n_samples]
              Class labels
    
    w : numpy array, shape = [n_samples]
              boosting like sample weights
    N: int
        num of samples in training data

    Returns
    ----------
    Cmin= estimator with the mininal training error
    
    min_error : int
        weighted error of Ct on training data
    
    """
    
    
    errors = []
    for c in range(len(C)):
        errors.append(WeightedError(C[c], X, y, w, N))
    min_error= min(errors)
    return C[errors.index(min_error)], min_error


def reset_weights(n):
    """
    set the sample weights with equal weights

    Parameters
    ----------
    n: int
        num of samples in training data

    Returns
    ----------
    w: numpy array, shape = [n_samples]
        Sample weights
    
    """
    return np.ones(shape=n) / n


In [ ]:
class BoostingBasedPruning:
    """
    Prune Bagging Ensembles by Boosting.

    Parameters
    ----------

    T : `int`
      number of classifiers.
    U : `int`
      number of classifiers to select from T [0,100].
    weights : `list` (default: `None`)
      If `None`, the majority rule voting will be applied to the predicted class labels.
        If a list of weights (`float` or `int`) is provided, the averaged raw probabilities (via `predict_proba`)
        will be used to determine the most confident class label.

    """

    def __init__(self, T, U, weights=None):
        self.T = T
        self.U = U
        self.weights = weights
        self.pool_d = []
        self.selected= math.ceil(self.T * self.U / 100)

    def fit(self, X, y):
        """
          build the estimators based on train data.

          Parameters
          ----------

          X : numpy array, shape = [n_samples, n_features]
              Training data
          y : list or numpy array, shape = [n_samples]
              Class labels
        """
        
        X = np.float64(X)
        y= np.array(y)
        pool_c= []

        for t in range(self.T):
            bg = BaggingClassifier(random_state=(t + 123), n_estimators=1,base_estimator=DecisionTreeClassifier(min_samples_leaf = 15)).fit(X, y)
            pool_c.append(bg)
        N = len(y)
        w = reset_weights(N)
        u = 0
        
        while u < self.selected:
            flag = True
            Du,eu = SelectBest(pool_c, X, y, w, N)
            if (eu > 0.5):
                w = reset_weights(N)
                #u = u - 1
                flag = False
            if (flag):
                u += 1
                self.pool_d.append(Du)
                pool_c.remove(Du)
            #update weights    
            y_pred= Du.predict(X)
            w=np.where(y != y_pred, w/(2 * eu), w/(2 * (1 - eu)))    
            
    def predict(self, X):
        """
          Parameters
          ----------

          X : numpy array, shape = [n_samples, n_features]

          Returns
          ----------

          maj : list or numpy array, shape = [n_samples]
              Predicted class labels by majority rule

        """
        
        self.classes_ = np.asarray([self.pool_d[m].predict(X) for m in range(self.selected)])

        if self.weights:
            avg = self.predict_proba(X)

            maj = np.apply_along_axis(lambda x: max(enumerate(x), key=operator.itemgetter(1))[0], axis=1, arr=avg)

        else:
            maj = np.asarray([np.argmax(np.bincount(self.classes_[:, c])) for c in range(self.classes_.shape[1])])

        return maj

    def predict_proba(self, X):

        """
        Parameters
        ----------

        X : numpy array, shape = [n_samples, n_features]

        Returns
        ----------

        avg : list or numpy array, shape = [n_samples, n_probabilities]
            Weighted average probability for each class per sample.

        """
        self.probas_ = np.asarray([self.pool_d[m].predict_proba(X) for m in range(self.selected)])
        avg = np.average(self.probas_, axis=0, weights=self.weights)

        return avg
        

    def get_params(self, deep=True):
        """
        Get parameters for this estimator.
        
        Parameters
        ----------
        deep : bool, default=True
            If True, will return the parameters for this estimator and
            contained subobjects that are estimators.
        
        Returns
        -------
        params : mapping of string to any
            Parameter names mapped to their values.
        """
        return {"T": self.T, "U": self.U}

    def set_params(self, **parameters):
        """
        Set the parameters of this estimator.
        The method works on simple estimators as well as on nested objects
        (such as pipelines). The latter have parameters of the form
        ``<component>__<parameter>`` so that it's possible to update each
        component of a nested object.
        Parameters
        ----------
        **params : dict
            Estimator parameters.
        Returns
        -------
        self : object
            Estimator instance.
        """
        for parameter, value in parameters.items():
          setattr(self, parameter, value)
        return self
      

# Part C

In [ ]:
space_xgb = {
    'max_depth' : hp.choice('max_depth', range(5, 7, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 205, 5)),
    'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
}

In [ ]:
space_bb = {
    'T' : hp.choice('T', range(10,150,20)),
    'U' : hp.choice('U', range(10,100,20)),
}

In [ ]:
def objective_xgb(params):
  """
        objective for Hyperopt tuning of XGBoost (based on sklearn), metric to minimaize
        
        Parameters
        ----------

        params : dict
          tuning parameters

        Returns
        ----------

        loss : float
            1-avarage of 3 folds accuracy

      """
    params = {'max_depth': int(params['max_depth']),
              'learning_rate': params['learning_rate'],
              'n_estimators': int(params['n_estimators']),
              'gamma': params['gamma'],
               'min_child_weight': params['min_child_weight'],
              'tree_method': 'gpu_hist'
                        }
    clf = XGBClassifier(**params)
    best_score = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=3, n_jobs=-1).mean()
    loss = 1 - best_score
    return loss

In [ ]:
def objective_bb(params):
    """
        objective for Hyperopt tuning of Boosting Based Pruning, metric to minimaize
        
        Parameters
        ----------

        params : dict
          tuning parameters

        Returns
        ----------

        loss : float
            1-avarage of 3 folds accuracy

      """


    kf=StratifiedKFold(n_splits=3, random_state= 43)
    metric=[] # for accuracy
    for train_index, test_index in kf.split(X,y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        y_train=y_train.tolist()
        clf = BoostingBasedPruning(**params)
        clf.fit(X_train,y_train)
        predictions=clf.predict(X_test)
        accuracy_score=metrics.accuracy_score(y_test,predictions) #Return the mean accuracy on the given test data and labels.
        metric.append(accuracy_score)
    best_score= np.mean(metric)
    loss= 1-best_score
    return loss

In [6]:
def hp_tuning(model):
    """
        Bayesian Hyperparameter Tuning with Hyperopt
        
        Parameters
        ----------

        model : str
          model to tuning the parameters

        Returns
        ----------

        best_params : dict
            tuning parameters

      """

    if model== 'XGBClassifier':
        objective= objective_xgb
        space=space_xgb
    else:
        objective= objective_bb
        space=space_bb
    
    trials = Trials()
    best_result = fmin(
    fn=objective,
    space=space,
    max_evals=50,
    rstate=np.random.RandomState(42),
    algo=tpe.suggest)
    best_barams=hyperopt.space_eval(space,best_result)
    return best_barams


In [5]:
def inference(X_test,model):
    """
        calaulate the time of predicting scaled 1000 samples
        
        Parameters
        ----------

        X_test : numpy array
            shape = [n_samples, n_features]

        model: object
            trained classifier

        Returns
        ----------

        inference_time : float
            time to predict 1000 samples

      """
    if (len(X_test) >= 1000):
        X_test=X_test.iloc[1:1000]
    n_samples= X_test.shape[0]
    start_inference=time.time()
    predictions_inference=model.predict(X_test)
    stop_inference=time.time()
    inference_time=(stop_inference-start_inference)*(1000/n_samples)
    return inference_time

In [ ]:
def FPR(y_true,y_prediction):
    """
        Evalute the model performance by FPR metirc

        Parameters
        ----------

        y_true : list or numpy array, shape = [n_samples]
              Class labels

        y_prediction : list or numpy array, shape = [n_samples]
              Predicted class labels by model

        Returns
        ----------

        FPR : float
            avarage of false positive rate

      """
    cnf_matrix = confusion_matrix(y_true, y_prediction)

    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    #false positive rate
    FPR = FP.sum()/(FP.sum()+TN.sum()) #micro average
    return FPR

In [3]:
def get_model(model_name, best_params):

    """
    create classifier model.
    
    Parameters
    ----------
    model_name : str
        name of classifier model
    
    best_params : dict
      parameters to define the model
    
    Returns
    -------
    model : object
      insatance of require model
    """
    if model_name== 'XGBClassifier':
        deafult = dict(
        tree_method='gpu_hist'
        )
        deafult.update(best_params)
        model= XGBClassifier(**deafult)
    else:
        model=BoostingBasedPruning(**best_param)
    return model
        

In [8]:
def preprocess(X,y):
  """
    Handaling missing values, catergorial features.

    Parameters
    ----------

    X : numpy array, shape = [n_samples, n_features]
        Training data
    y : list or numpy array, shape = [n_samples]
        Class labels
  """
  
  #missing values
  for column in X.columns:
    X[column].fillna(X[column].mode()[0], inplace=True) 
  #one hot encoder for X
  X= pd.get_dummies(X)
  #label encoder to target
  le = preprocessing.LabelEncoder()
  y= pd.Series(le.fit_transform(y),index= y.index)

  return X,y


In [ ]:

def Part_C(db_path):

  #params
  average='micro' #avarage metric for multiclass classification
  seed = 123

  #create dataframe to compare between algorithams
  df = pd.DataFrame(columns=['dataset_name','algorithm_name','cross_validation','hyper_parameters_values','accuracy','tpr','fpr','precision','auc','pr_curve','training_time','inference_time'])
  kf=StratifiedKFold(n_splits=10, random_state= seed) # for 10fold cross validation


  path= db_path
  databases_path = os.listdir(path)
  alg_list=['XGBClassifier','BoostingBasedPruning']

  for database_path in databases_path:
      if not database_path in db_list:
        for model_name in alg_list:
            dataset= pd.read_csv(os.path.join(path, database_path))
            dataset_name=database_path 
            
            cross_validation_num=0 # for result table
            # split data into X and y
            X = dataset.iloc[:,:-1] 
            y = dataset.iloc[:,-1] #last columns- clss
          
            X,y= preprocess(X,y)
            # split data into train and test sets by cross validation 10 folds
            for train_index, test_index in kf.split(X,y):
                cross_validation_num=cross_validation_num+1
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                best_param=hp_tuning(model_name) #tuning the parameters of model
                start_training=time.time() #calculate training time
                model= get_model(model_name,best_param)
                clf = OneVsRestClassifier(model).fit(X_train,y_train) #training by OVR prucedure for multiclass classification
                stop_training=time.time()
                predictions=clf.predict(X_test) #predict the class
                y_score= clf.predict_proba(X_test) #predict the probabilities to class
                
                #model evaluation 
                accuracy_score=metrics.accuracy_score(y_test,predictions) 
                precision=precision_score(y_test,predictions,average=average) 
                tpr =metrics.recall_score(y_test, predictions, average=average)
                fpr=FPR(y_test,predictions)
                
                classes= y.unique()
                y_binary = label_binarize(y_test, classes=range(0,len(y_score[0]),1)) #transform y to binary
                if(len(classes)<=2): #for binary classification
                    y_score= y_score[:,1]
                    y_binary= y_test
                auc=roc_auc_score(y_binary, y_score, multi_class="ovr",average=average) 
                pr_curve=average_precision_score(y_binary, y_score, average=average) 
                
                training_time=stop_training-start_training
                inference_time=inference(X_test,clf) #time to predict 1000 samples
                df=df.append({
                'dataset_name':dataset_name,'algorithm_name':model_name,'cross_validation':cross_validation_num,'hyper_parameters_values':best_param,'accuracy':accuracy_score
                    ,'tpr':tpr,'fpr':fpr,'precision':precision,'auc': auc,'pr_curve':pr_curve,'training_time':training_time,'inference_time':inference_time
                  },ignore_index=True)
            
            #save the DataFrame
            file_name= str(model_name)+"_results.csv"
            df.to_csv(file_name,index=False)
            

# Part D

In [19]:
def Part_D(xgb_data,bb_data):
  auc_xgb=xgb_data.groupby(['dataset_name'], as_index= False)['auc'].mean() 
  auc_bb=bb_data.groupby(['dataset_name'], as_index= False)['auc'].mean()

  stat, p =wilcoxon(auc_bb['auc'],auc_xgb['auc'], alternative='two-sided') #regect null hypothesis 
  print('Statistics=%.0f, p=%.3f' % (stat, p))
  stat, p= wilcoxon(auc_xgb['auc'],auc_bb['auc'], alternative='greater')
  print('Statistics=%.0f, p=%.3f' % (stat, p))

Statistics=3890, p=0.002
Statistics=7136, p=0.001


# Part E

In [ ]:
def prapare_data(xgb_data,bb_data,meta_feature):
    """
    Compare AUC performace between XGBoost and BB and build unified training data

    Parameters
    ----------

    xgb_data : numpy array, shape = [10*n_datasets, n_features]
        results of XGBoost on datasets with 10 CV
    bb_data : numpy array, shape = [10*n_datasets, n_features]
        results of BB on datasets with 10 CV

    Return
    ----------

    data : numpy array, shape = [n_datasets, n_meta_features]
        Training data for meta learning
  """
  

  # calculating avarage auc to dataset
  auc_xgb=xgb_data.groupby(['dataset_name'], as_index= False)['auc'].mean() 
  auc_bb=bb_data.groupby(['dataset_name'], as_index= False)['auc'].mean()

  #compare between xgb to bb by auc
  comparison= pd.merge(auc_xgb, auc_bb, on='dataset_name')
  comparison['target']= np.where(comparison['auc_x'] > comparison['auc_y'],0,1) # '0' for xgb and '1' for bb
  #Handling incompatible values
  comparison['dataset_name']=comparison['dataset_name'].replace({'abalon.csv': 'abalone.csv','pittsburg-bridges-T-OR-D_R.csv':'pittsburg-bridges-T-OR-D.csv','statlog-heart_.csv':'statlog-heart.csv','wine-quality-red.csv':'wine_quality_red.csv'})
  comparison['dataset']= comparison['dataset_name'].apply(lambda x: x.split(".")[0]) #remove '.csv'

  data= pd.merge(meta_feature,comparison[['target','dataset']], on='dataset', how='outer')
  return data

In [10]:
def feature_importance(xgb):
  """
    save feature importance by several parameters and plot

    Parameters
    ----------

    xgb : Trained meta learning model

  """
  

  param = {'weight','gain','cover'}
  for char in param:
      name=str(char) + '.csv'
      ans=xgb.get_booster().get_score(importance_type= char)
      with open(name, 'w') as f:  # Just use 'w' mode in 3.x
          w = csv.DictWriter(f, ans.keys())
          w.writeheader()
          w.writerow(ans)

  plot_importance(xgb, max_num_features=10) # top 10 most important features
  plt.show()

In [9]:
def shap (X_test,xgb):
  """
    Plot shap values and summary

    Parameters
    ----------
    X : numpy array, shape = [n_samples, n_features]
        Test data

    xgb : Trained meta learning model

  """
  # Create object that can calculate shap values
  explainer = shap.TreeExplainer(xgb)
  row_to_show = 10
  data_for_prediction = X_test.iloc[row_to_show]
  # Calculate Shap values
  shap_values = explainer.shap_values(data_for_prediction)
  shap.initjs()
  shap.force_plot(explainer.expected_value, shap_values[0], data_for_prediction)

  # Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
  shap_values = explainer.shap_values(X_test)

  # Make plot. Index of [1] is explained in text below.
  shap.summary_plot(shap_values, X_test ,max_display=5)

  # make plot
  features= ['instances','f120','f123','f124','Features.Mean.StandardDeviation']
  for feature in features:
      shap.dependence_plot(feature, shap_values, X_test, interaction_index=None)


In [ ]:


def Part_E (xgb_data,bb_data,meta_feature):
  
  #-------------Init-------------
  #define metircs to save
  meta_features_output = pd.DataFrame(columns=['dataset_name','algorithm_name','cross_validation','hyper_parameters_values','accuracy','tpr','fpr','precision','auc','pr_curve','training_time','inference_time'])
  #for calculate avarage metrics
  output=pd.DataFrame(columns=['y_test','predictions','y_score']) 
  average='micro'

  #-------------Get Data-------------
  data= prapare_data(xgb_data,bb_data,meta_feature)
  y=data['target']
  X=data.iloc[:,1:-1]
  X, y= preprocess(X,y)
  
  #-------------Leave One Out#-------------
  loo=LeaveOneOut()
  loo.get_n_splits(X)
  for train_index, test_index in loo.split(X):
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      start_training=time.time()
      xgb = XGBClassifier().fit(X_train,y_train)
      stop_training=time.time()    
      prediction=xgb.predict(X_test)
      y_score= xgb.predict_proba(X_test)
      training_time=stop_training-start_training
      inference_time=inference(X_test,xgb)
      output=output.append({'y_test':y_test.to_list(),'predictions':prediction,'y_score':y_score[:,1],'training_time':training_time,'inference_time':inference_time},ignore_index=True)


  ##-------------Results-------------
  accuracy_score=metrics.accuracy_score(output['y_test'].to_list(),output['predictions'].to_list()) #Return the mean accuracy on the given test data and labels.
  precision=precision_score(output['y_test'].to_list(),output['predictions'].to_list(),average=average)
  tpr =metrics.recall_score(output['y_test'].to_list(), output['predictions'].to_list(), average=average)
  fpr=FPR(output['y_test'].to_list(),output['predictions'].to_list())
  auc=roc_auc_score(output['y_test'].to_list(), output['y_score'].to_list(), multi_class="ovr",average=average) 
  pr_curve=average_precision_score(output['y_test'].to_list(), output['y_score'].to_list(), average=average)  
  mean_training_time=output['training_time'].mean()
  mean_inference_time=output['inference_time'].mean()
  meta_features_output=meta_features_output.append({'accuracy':accuracy_score
  ,'tpr':tpr,'fpr':fpr,'precision':precision,'auc': auc,'pr_curve':pr_curve,'training_time':mean_training_time,'inference_time': mean_inference_time
  },ignore_index=True)
  #save the DataFrame
  meta_features_output.to_csv('meta_output.csv',index=False)
    
  #-------------Shap&Feature Importance------------- 
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1,random_state=123)
  xgb =XGBClassifier().fit(X_train,y_train)
  shap (X_test,xgb)
  feature_importance(xgb)
  

#Main

In [5]:
db_path='./classification_datasets'
xgb_path= './all_xgb.csv'
bb_path= './bb_all_final.csv'
meta_feature_path=  './ClassificationAllMetaFeatures.csv'

xgb_data=pd.read_csv(xgb_path) 
bb_data=pd.read_csv(bb_path) 
meta_feature= pd.read_csv(meta_feature_path)

Part_C(db_path)
Part_D(xgb_data,bb_data)
Part_E(xgb_data,bb_data,meta_feature)